In [1]:
import tensorflow as tf
import tensorflow_hub as hub
# from google.colab import drive
# drive.mount("/content/drive")

2022-06-01 20:24:12.550929: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-01 20:24:12.551018: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
print(tf.__version__)

2.8.0


## This is my own dataset (i collect it from my whatsapp chat screenshot and from my personal gallery), so if you want to use it please reach me in Discord Amna#3650

In [4]:
!unzip /content/drive/MyDrive/own_dataset/chat-non-chat.zip

Archive:  /content/drive/MyDrive/own_dataset/chat-non-chat.zip
   creating: chat/
  inflating: chat/telegram-channel-vs-group-5_935adec67b324b146ff212ec4c69054f.png  
  inflating: chat/w1200.jpeg         
  inflating: chat/7f819df55211e9a7e4.jpeg  
  inflating: chat/f3ebf15475f5ac1f66.jpeg  
  inflating: chat/motorist-5-telegram-chats-drivers-should-join-sghitch.jpg  
  inflating: chat/882118_3.png       
  inflating: chat/1_x6mcLJpMPEkDSXhe2Hvkqw.png  
  inflating: chat/Telegram-Edit-Chat-History.jpg  
  inflating: chat/c0d263ce79.png     
  inflating: chat/motorist-5-telegram-chats-drivers-should-join-sgrdinformers-group.jpeg  
  inflating: chat/1_EZHzd9e3mp3If037Ui33mw.png  
  inflating: chat/5bd7c8f4708afe28f8.jpeg  
  inflating: chat/IMG_B068DF967AC1-1.jpeg  
  inflating: chat/IMG_6821.jpg       
  inflating: chat/sgmrt-530x1024.jpg  
  inflating: chat/sms-spam.jpg.crdownload  
  inflating: chat/VOZxo.png          
  inflating: chat/sms-spam-dan-penipuan.jpg  
  inflating: chat/Fa

In [5]:
!mkdir data
!cp -r chat data/
!cp -r non data/

In [37]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'data', 
        target_size=(224, 224), 
        batch_size=64,
        class_mode='binary')


Found 1046 images belonging to 2 classes.


In [36]:
def make_model(base_model):
    finetuning_model = tf.keras.Sequential([
        tf.keras.layers.InputLayer((224, 224, 3)),
        base_model,
        # tf.keras.layers.GlobalAveragePooling2D(),
        # tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # tf.keras.layers.Dense(32, activation="relu"),
        # tf.keras.layers.Dropout(0.1),
        # tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    return finetuning_model

# base_layer = hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5", trainable=False)

base_layer = hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/classification/2", trainable=False)

model = make_model(base_layer)


model.build(input_shape=(None, 224, 224, 3))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_7 (KerasLayer)  (None, 1000)              7200312   
                                                                 
 flatten_6 (Flatten)         (None, 1000)              0         
                                                                 
 dense_26 (Dense)            (None, 64)                64064     
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_27 (Dense)            (None, 1)                 65        
                                                                 
Total params: 7,264,441
Trainable params: 64,129
Non-trainable params: 7,200,312
_________________________________________________________________


In [39]:
model.compile(
    optimizer="adam",
    loss= "binary_crossentropy",
    metrics=["accuracy"]    
)

print("Training model")
model.fit(
    train_generator,
    epochs=5
)

Training model
Epoch 1/5
 9/17 [==============>...............] - ETA: 19s - loss: 0.2622 - accuracy: 0.8951

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


17/17 [==============================] - 44s 2s/step - loss: 0.2585 - accuracy: 0.8929
Epoch 2/5
17/17 [==============================] - 42s 2s/step - loss: 0.2021 - accuracy: 0.9302
Epoch 3/5
17/17 [==============================] - 42s 2s/step - loss: 0.2168 - accuracy: 0.9187
Epoch 4/5
17/17 [==============================] - 42s 2s/step - loss: 0.2017 - accuracy: 0.9235
Epoch 5/5
17/17 [==============================] - 42s 2s/step - loss: 0.1443 - accuracy: 0.9436


In [40]:
model.save("/content/drive/MyDrive/chat/saved_models/")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/chat/saved_models/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/chat/saved_models/assets


In [4]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a non chat")
  else:
    print(fn + " is a chat")
 

Saving chat1.jpg to chat1.jpg
Saving chat2.jpeg to chat2.jpeg
Saving chat3.jpg to chat3.jpg
Saving nchat1.jpg to nchat1.jpg
Saving nchat2.jpeg to nchat2.jpeg
Saving nchat3.jpeg to nchat3 (1).jpeg
[0.16060042]
chat1.jpg is a chat
[0.9940798]
chat2.jpeg is a non chat
[0.2729307]
chat3.jpg is a chat
[0.9328035]
nchat1.jpg is a non chat
[0.9852457]
nchat2.jpeg is a non chat
[0.9969702]
nchat3.jpeg is a non chat


In [2]:
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model("saved_models/")

model.summary()

2022-06-01 20:27:11.010411: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-01 20:27:11.010509: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-01 20:27:11.010554: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (uwu): /proc/driver/nvidia/version does not exist
2022-06-01 20:27:11.102308: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_7 (KerasLayer)  (None, 1000)              7200312   
                                                                 
 flatten_6 (Flatten)         (None, 1000)              0         
                                                                 
 dense_26 (Dense)            (None, 64)                64064     
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_27 (Dense)            (None, 1)                 65        
                                                                 
Total params: 7,264,441
Trainable params: 64,129
Non-trainable params: 7,200,312
_________________________________________________________________


In [51]:
path = "/home/vannyezha/dataset/IMG_20220601_211907.jpg"

img = tf.keras.preprocessing.image.load_img(path,color_mode="rgb", target_size=(224, 224))
x = tf.keras.preprocessing.image.img_to_array(img)
x /= 255

x = np.expand_dims(x, axis=0)
images = np.vstack([x])

# # 
classes = model.predict(images, batch_size=10)
print(str(classes[0][0]))

1/1 [==============================] - 0s 77ms/step
0.094402626


In [16]:
from PIL import Image